In [1]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 140 kB/s 


In [2]:
import faiss

In [3]:
ngpus = faiss.get_num_gpus()
print("number of GPUs:", ngpus)

number of GPUs: 1


In [4]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/17666')

Mounted at /content/drive


In [5]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.metrics import classification_report, confusion_matrix


questionData = pd.read_csv('/content/drive/MyDrive/17666/test_question.csv')
questionData.head(10)

,id,question
0,1,能否提供付款帳戶?
1,1,客戶要付款，請提供銀行帳號
2,1,銀行資料可以找誰拿?
3,1,有最新的銀行信息文件嗎？
4,1,幫忙確認一下我司銀行帳戶對不對
5,1,請提供開戶銀行&銀行賬號
6,2,客戶錯匯款項到其他公司別帳戶，是否可以轉款沖帳？
7,2,大聯大商貿的貨款客戶付到大聯大商貿深圳，可以辦轉款嗎？
8,2,友尚上海的貨款客戶付到大聯大商貿，怎麼辦？
9,2,客戶匯錯，可以沖帳嗎?


In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 16.2 MB/s 
     |████████████████████████████████| 6.6 MB 57.9 MB/s 
     |████████████████████████████████| 120 kB 51.6 MB/s 


In [7]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.4 MB/s 
     |████████████████████████████████| 1.3 MB 36.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=bc8a510d095754ea429cd4ed32c32a52125085ab0c25253026ee2d5f04fbc5a5
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [8]:
import torch
from sentence_transformers import SentenceTransformer
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/555 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/505 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

cuda:0


In [9]:
 # 1. init faiss index
#index = faiss.index_factory(768,"IDMap,Flat,L2norm",faiss.METRIC_INNER_PRODUCT)

In [10]:
# Convert question to vectors
embeddings = model.encode(questionData.question.to_list(), show_progress_bar=True)

# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, questionData.id.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Number of vectors in the Faiss index: 496


In [11]:
print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 496


In [12]:
# Paper abstract
questionData.iloc[4, 1]

'幫忙確認一下我司銀行帳戶對不對'

In [13]:
faiss.write_index(index,"{}.index".format('faiss_index'))

In [24]:
query = [" ".join( [ word for word in '詢證函要找誰處理?【商貿上海】' ])]
print("bert-serving input: ",query)

#若立帳日為1年內的暫收/銷折款，可拋電子傳簽單申請轉回帳上，說明暫收款轉回原因、提供文件編號、報備使用超過六個月暫收款。。
#簽核層級：業務端>部>處>群>Jessica>會計>帳管師cc直屬主管※立帳日超過一年以上請先洽詢帳管師

bert-serving input:  ['詢 證 函 要 找 誰 處 理 ? 【 商 貿 上 海 】']


In [25]:
from functools import reduce
vectors = model.encode(query)
#vectors = normaliz_vec(vectors.tolist())
vecList=vectors.tolist()

for i in range(len(vecList)):
        vec = vecList[i]
        square_sum = reduce(lambda x, y: x+y, map(lambda x: x*x, vec))
        sqrt_square_sum = np.sqrt(square_sum)
        coef = 1/sqrt_square_sum
        vec = list(map(lambda x: x*coef, vec))
        vecList[i] = vec



query_list = np.array(vectors).astype('float32')
dis,ind = index.search( query_list,k=5 )

In [26]:


list_ans = dis.tolist;
# print('first = ', list_ans)
print('dis = ', dis.flatten().tolist())
print('ind = ',ind)

print('dis type= ', type(dis[0][0]))
print('dis type= ', np.format_float_positional(np.float32(dis[0][0])))




'''
>>> np.format_float_positional(np.float32(np.pi))
'3.1415927'
>>> np.format_float_positional(np.float16(np.pi))
'3.14'
>>> np.format_float_positional(np.float16(0.3))
'0.3'
>>> np.format_float_positional(np.float16(0.3), unique=False, precision=10)
'0.3000488281'
'''


        

dis =  [6.124017265518589e-11, 1.0354751348495483, 37.07257080078125, 64.00701141357422, 76.59418487548828]
ind =  [[ 13 121   8  11 133]]
dis type=  <class 'numpy.float32'>
dis type=  0.00000000006124017


"\n>>> np.format_float_positional(np.float32(np.pi))\n'3.1415927'\n>>> np.format_float_positional(np.float16(np.pi))\n'3.14'\n>>> np.format_float_positional(np.float16(0.3))\n'0.3'\n>>> np.format_float_positional(np.float16(0.3), unique=False, precision=10)\n'0.3000488281'\n"

In [27]:
answerData = pd.read_csv('/content/drive/MyDrive/17666/test_answer.csv')
answerData.head(10)

,id,answer
0,1,請諮詢財務對應帳戶訊息
1,2,因不同公司別分屬不同帳本，需明確向客戶說明，並請客戶付款至正確帳戶。如有其他問題，請諮詢財務
2,3,請提供水單給財務查款；Oracle也可查詢財務是否掛帳
3,4,依各銀行流程作業時間而定；台幣約1~2天到帳；美金約 2~3天不等；亦可請客戶提供付款水單給...
4,5,USD 50以上\nTWD 依公訂手續費標準\n且手續費不能超過該筆收款的2%
5,6,若客戶自行付款且銀行信息有體現客戶名稱，財務會直接掛帳\n若客戶委託第三方付款，則須提供水單...
6,7,請諮詢會計
7,8,若立帳日為1年內的暫收/銷折款，可拋電子傳簽單申請轉回帳上，說明暫收款轉回原因、提供文件編號...
8,9,帳管師無權處理，請業務申請FLOW
9,10,商貿收款掛在總帳，需核銷AR後才會轉入各OU


In [28]:
top_number=1
for id in ind[0]:
  print("*top {}".format(top_number))
  print("距離差異: {}".format(np.format_float_positional(np.float32(dis[0][top_number-1]))))
  print("id: {}".format(id))
  filt = (answerData['id'] == id)
  print("ANS: {}".format(answerData[filt].answer))
  print("\n")
  top_number =top_number+1

*top 1
距離差異: 0.00000000006124017
id: 13
ANS: Series([], Name: answer, dtype: object)


*top 2
距離差異: 1.0354751
id: 121
ANS: Series([], Name: answer, dtype: object)


*top 3
距離差異: 37.07257
id: 8
ANS: 7    若立帳日為1年內的暫收/銷折款，可拋電子傳簽單申請轉回帳上，說明暫收款轉回原因、提供文件編號...
Name: answer, dtype: object


*top 4
距離差異: 64.00701
id: 11
ANS: Series([], Name: answer, dtype: object)


*top 5
距離差異: 76.594185
id: 133
ANS: Series([], Name: answer, dtype: object)




In [19]:
# 4. text search (topK = 5)